In [235]:
import numpy as np
import torch
from baselines.cifar10_loader import load_and_process_cifar
from models.classifier import MLP
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
class CoolDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        i = self.labels[idx]
        l = torch.zeros(size=(10,))
        l[i] = 1
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long).squeeze(-1)#l

In [237]:
x_train, x_val, x_test, y_train, y_val, y_test = load_and_process_cifar(n_components=512, return_val=True)


PCA explained variance ratio:  1.0000000000000004


In [238]:
x_train1, x_train2 = x_train[:40000], x_train[40000:]
y_train1, y_train2 = y_train[:40000], y_train[40000:]

In [239]:
base_model = MLP(input_size=512,
    hidden_layers=[256, 64],
    output_size=10,
    dropout=0.5,
    logger='auc', loss='ce',
    loss_params=None, optim='adam',
    optim_params=dict(lr=1e-3, weight_decay=0),
    scheduler=None,
    scheduler_params=None,
    legacy=True,
)

In [240]:
batch_size = 64
learning_rate = 1e-4
epochs=50

In [241]:
trainer = pl.Trainer(
    limit_train_batches=100,
    max_epochs=epochs
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [243]:
train_loader = DataLoader(CoolDataset(x_train1, y_train1), batch_size=batch_size, num_workers=20)
val_loader = DataLoader(CoolDataset(x_train2, y_train2), batch_size=batch_size, num_workers=20)
test_loader = DataLoader(CoolDataset(x_test, y_test), batch_size=batch_size, num_workers=20)

In [244]:
trainer.fit(base_model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 148 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
2 | _logger | AUCLogger        | 0      | train
-----------------------------------------------------
148 K     Trainable params
0         Non-trainable params
148 K     Total params
0.594     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/home/viet/anaconda3/envs/dl/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [245]:
trainer.save_checkpoint('pddm_cifar10.pth')

In [246]:
model2 = MLP.load_from_checkpoint('pddm_cifar10.pth', input_size=512,
    hidden_layers=[256, 64],
    output_size=10,
    dropout=0.5,
    logger='auc', loss='ce',
    loss_params=None, optim='adam',
    optim_params=dict(lr=1e-3, weight_decay=0),
    scheduler=None,
    scheduler_params=None,
    legacy=True)

In [247]:
# sanity check

def evaluate(model, loader):
    model.eval()
    model = model.cuda()
    count = 0
    acc = 0 
    for features, labels in loader:
        features, labels = features.cuda(), labels.cuda()
        logits = model(features)
        output = F.softmax(logits)
        preds = torch.argmax(output, dim=-1)
        acc += torch.sum(preds == labels)
        count += len(labels)
    return acc / count

In [249]:
evaluate(model2, test_loader)

/tmp/ipykernel_97119/3211225906.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(logits)


tensor(0.6740, device='cuda:0')

In [250]:
import pickle

def save_dataset(dataset, name):
    with open(name, 'wb') as f:
        pickle.dump(dataset, f)

In [251]:
dct = {
    'x_train1': x_train1,
    'x_train2': x_train2,
    'x_val': x_val,
    'x_test': x_test,
    'y_train1':y_train1,
    'y_train2': y_train2,
    'y_val': y_val,
    'y_test': y_test
}

In [252]:
save_dataset(dct, 'pddm_cifar10.pkl')

In [253]:
with open('pddm_cifar10.pkl', 'rb') as f:
    dct = pickle.load(f)